In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Flatten,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard,ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [2]:
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
y_train.shape

(60000,)

In [5]:
x_test.shape

(10000, 28, 28)

In [6]:
y_test.shape

(10000,)

In [7]:
# Normalize pixel values to [0, 1] and reshape for CNN input
def preprocess(images):
    images = images / 255.0
    return images.reshape(-1, 28, 28, 1)

In [8]:
x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [9]:
x_train.shape

(60000, 28, 28, 1)

In [10]:
x_test.shape

(10000, 28, 28, 1)

In [11]:
# Define an ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=10,       # Randomly rotate images by up to 10 degrees
    width_shift_range=0.1,   # Randomly shift images horizontally by 10% of the width
    height_shift_range=0.1,  # Randomly shift images vertically by 10% of the height
    zoom_range=0.1,          # Randomly zoom in/out by up to 10%
    horizontal_flip=False    # Do not flip images horizontally (not meaningful for digits)
)

In [12]:
# Create a generator for augmented data
train_generator = datagen.flow(x_train, y_train, batch_size=100)

In [13]:
model = Sequential(
    [
        Conv2D(32,(3,3),activation='relu',padding='same',input_shape=(28,28,1)),
        MaxPool2D((2,2)),
        BatchNormalization(),
        Conv2D(32,(3,3),activation='relu',padding='same'),
        MaxPool2D((2,2)),
        Conv2D(64,(3,3),activation='relu',padding='same'),
        MaxPool2D((2,2)),
        BatchNormalization(),
        Conv2D(64,(3,3),activation='relu',padding='same'),
        MaxPool2D((2,2)),
        BatchNormalization(),
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ]
)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 14, 14, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                        

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [16]:
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [17]:
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [18]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience = 5, restore_best_weights=True)

In [19]:
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3)

In [20]:
# Training the model
history = model.fit(
    train_generator, epochs=100, 
    validation_data=(x_test, y_test),
    callbacks = [tensorboard_callback,early_stopping_callback,reduce_lr]
)

Epoch 1/100
600/600 [==============================] - 40s 64ms/step - loss: 0.1822 - accuracy: 0.9432 - val_loss: 0.0715 - val_accuracy: 0.9772 - lr: 0.0010
Epoch 2/100
600/600 [==============================] - 35s 59ms/step - loss: 0.0654 - accuracy: 0.9797 - val_loss: 0.0553 - val_accuracy: 0.9825 - lr: 0.0010
Epoch 3/100
600/600 [==============================] - 36s 60ms/step - loss: 0.0540 - accuracy: 0.9829 - val_loss: 0.0931 - val_accuracy: 0.9693 - lr: 0.0010
Epoch 4/100
600/600 [==============================] - 35s 58ms/step - loss: 0.0462 - accuracy: 0.9856 - val_loss: 0.0619 - val_accuracy: 0.9807 - lr: 0.0010
Epoch 5/100
600/600 [==============================] - 36s 60ms/step - loss: 0.0407 - accuracy: 0.9876 - val_loss: 0.0381 - val_accuracy: 0.9885 - lr: 0.0010
Epoch 6/100
600/600 [==============================] - 37s 61ms/step - loss: 0.0361 - accuracy: 0.9889 - val_loss: 0.0255 - val_accuracy: 0.9921 - lr: 0.0010
Epoch 7/100
600/600 [==============================]

In [22]:
# Loading Tensorboard Extension
%load_ext tensorboard

In [23]:
%reload_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit/ --port=8081

Reusing TensorBoard on port 8081 (pid 12616), started 0:00:03 ago. (Use '!kill 12616' to kill it.)

In [25]:
model.save('model.h5')

c:\Users\azefr\OneDrive\Desktop\Projects\CNN\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
